# NBA Predictive Analysis

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from elopy import *
import pandas as pd
imp = Implementation()

In [2]:
# NBA season we will be analyzing
years = [2020]
months = ['october', 'november', 'december', 'january', 'february', 'march']

headers = []
rows = []
match_date = []
match_stats = []
raw_match_data = []
sortedRanking = []

imp.addPlayer('Hank')

for year in years:
    for month in months:
        # URL page we will scraping
        url = "https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html".format(year, month)

        # this is the HTML from the given URL`
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')

        # use findALL() to get the column headers
        # Example -> soup.findAll('tr', limit=1), we use limit 1 to only get the first header

        # use getText() to extract the text we need into a list
        headers = [th.getText() for th in soup.findAll('tr', limit=1)[0].findAll('th')]
        unwanted_headers = ['\xa0', '\xa0','Notes']

        headers = [header for header in headers if header not in unwanted_headers]
        headers.insert(4, 'Result') #Adds Result Header
        headers.insert(5, 'Diff') #Adds Diff Header 
        # print(headers)

        # avoid the first header row -> [1:]
        rows = soup.findAll('tr')[1:]
        # print(rows)

        match_date = [[th.getText() for th in rows[i].findAll('th')]
                    for i in range(len(rows))]

        # print(game_date)

        match_stats = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]

        #     Deletes (by index) the unnecessary td concerning ['\xa0', '\xa0','Notes'] headers
        for i in range(len(rows)):
            del match_stats[i][5], match_stats[i][5],match_stats[i][6]

        # Adds Won or Lost value to Result Header
        for i in range(len(rows)):
            match_stats[i].insert(3, int(match_stats[i][2])- int(match_stats[i][4]))#Adds diff

            if int(match_stats[i][2]) > int(match_stats[i][5]):
                match_stats[i].insert(3,'Won')
            else: 
                match_stats[i].insert(3,'Lost')

        for i in range(len(rows)):
            imp.addPlayer(match_stats[i][1]) #Adds Team to Elo
            imp.addPlayer(match_stats[i][5]) #Adds Team to Elo

        # print (imp.getRatingList())   
        for i in range(len(rows)):
            if int(match_stats[i][2]) > int(match_stats[i][6]):
                imp.recordMatch(match_stats[i][1], match_stats[i][5],winner= match_stats[i][1])
            elif int(match_stats[i][2]) < int(match_stats[i][6]):
                imp.recordMatch(match_stats[i][1], match_stats[i][5],winner= match_stats[i][5])

        # print ("\n",imp.getRatingList())   

        sortedRanking = sortTuple((imp.getRatingList()))

        # Merge game _date and match stats into one list

        for i in range(len(match_date)):
            raw_match_data .append(match_date[i] + match_stats[i])

        # print(raw_match_data )


In [3]:
stats = pd.DataFrame(raw_match_data, columns = headers)
stats.head(1000)

,Date,Start (ET),Visitor/Neutral,PTS,Result,Diff,Home/Neutral,PTS,Attend.
0,"Tue, Oct 22, 2019",8:00p,New Orleans Pelicans,122,Lost,-8,Toronto Raptors,130,"20,787"
1,"Tue, Oct 22, 2019",10:30p,Los Angeles Lakers,102,Lost,-10,Los Angeles Clippers,112,"19,068"
2,"Wed, Oct 23, 2019",7:00p,Chicago Bulls,125,Lost,-1,Charlotte Hornets,126,"15,424"
3,"Wed, Oct 23, 2019",7:00p,Detroit Pistons,119,Won,9,Indiana Pacers,110,"17,923"
4,"Wed, Oct 23, 2019",7:00p,Cleveland Cavaliers,85,Lost,-9,Orlando Magic,94,"18,846"
5,"Wed, Oct 23, 2019",7:30p,Minnesota Timberwolves,127,Won,1,Brooklyn Nets,126,"17,732"
6,"Wed, Oct 23, 2019",7:30p,Memphis Grizzlies,101,Lost,-19,Miami Heat,120,"19,600"
7,"Wed, Oct 23, 2019",7:30p,Boston Celtics,93,Lost,-14,Philadelphia 76ers,107,"20,422"
8,"Wed, Oct 23, 2019",8:30p,Washington Wizards,100,Lost,-8,Dallas Mavericks,108,"19,816"
9,"Wed, Oct 23, 2019",8:30p,New York Knicks,111,Lost,-9,San Antonio Spurs,120,"18,354"


In [4]:
ranking = pd.DataFrame(sortedRanking, columns = ['Team', 'Rank'])
# ranking.head(len(rows))
# print(imp.predictMatch('Cleveland Cavaliers', 'Milwaukee Bucks'))